In [1]:
import time
from ultralytics import YOLO
import numpy as np
import imutils
import cv2

# Tipos de emociones del detector
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Cargamos imágenes de emociones
emotion_images = {
    'angry': cv2.imread('emociones/angry.jpg'),
    'disgust': cv2.imread('emociones/disgust.jpg'),
    'fear': cv2.imread('emociones/fear.jpg'),
    'happy': cv2.imread('emociones/happy.jpg'),
    'neutral': cv2.imread('emociones/neutral.jpg'),
    'sad': cv2.imread('emociones/sad.jpg'),
    'surprise': cv2.imread('emociones/surprise.jpg')
}

# Cargamos el modelo YOLO
emotion_model = YOLO("best.pt")

# Configuración de cámara
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Variables para controlar el tiempo de espera
last_emotion = None
last_detection_time = 0
emotion_delay = 2  # Segundos de espera antes de actualizar la emoción

def process_emotions(frame, model):
    results = model(frame, verbose=False)
    
    boxes = []
    emotions = []
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cls = int(box.cls[0])
            conf = box.conf[0].item()
            
            boxes.append((x1, y1, x2, y2))
            emotions.append((cls, conf))
    
    return boxes, emotions

while True:
    ret, frame = cam.read()
    if not ret:
        break
    
    frame = imutils.resize(frame, width=640)
    boxes, emotions = process_emotions(frame, emotion_model)
    
    right_panel = np.zeros((frame.shape[0], 300, 3), dtype=np.uint8)
    detected_emotion = None

    for box, (cls, conf) in zip(boxes, emotions):
        x1, y1, x2, y2 = box
        label = classes[cls]
        prob = conf * 100

        # Dibujar elementos en el frame
        cv2.rectangle(frame, (x1, y1-40), (x2, y1), (0, 100, 0), -1)
        cv2.putText(frame, f"{label}: {prob:.0f}%", (x1+5, y1-15),
                    cv2.FONT_HERSHEY_DUPLEX, 0.7, (50, 255, 50), 1)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        detected_emotion = label

    # Actualizar la emoción solo si ha pasado el tiempo de espera
    current_time = time.time()
    if detected_emotion and (last_emotion is None or detected_emotion != last_emotion):
        if current_time - last_detection_time > emotion_delay:
            last_emotion = detected_emotion
            last_detection_time = current_time

    # Mostrar imagen de la emoción detectada
    if last_emotion:
        emotion_image = emotion_images.get(last_emotion)
        if emotion_image is not None:
            emotion_image = cv2.resize(emotion_image, (300, 300))
            y_start = (right_panel.shape[0] - 300) // 2
            right_panel[y_start:y_start+300, 0:300] = emotion_image
            cv2.putText(right_panel, last_emotion.upper(), 
                        (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (50, 255, 50), 2)

    # Combinar los paneles
    composite_frame = cv2.hconcat([frame, right_panel])
    cv2.imshow("Detección de Emociones", composite_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Limpieza
cam.release()
cv2.destroyAllWindows()
